In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
from jellyfish.core import Client
from jellyfish.history_loader import load_candles_history
from jellyfish import utils
from datetime import datetime, timedelta

utils.disable_warnings()

pair = 'btcusdt'
mid_dt = datetime(year=2021, month=9, day=25)
size = 5000 * 2

frame = load_candles_history(Client(), pair, end_dt=mid_dt, interval='1m', candles_num=size)
val_frame = load_candles_history(Client(), pair, start_dt=mid_dt, interval='1m', candles_num=size)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [63]:
from jellyfish import indicator
import numpy as np
import pandas as pd

def preprocess(frame: pd.DataFrame):
    returns = frame.Close / frame.Open
    hurst = indicator.hurst(returns, kind=indicator.HURST_CHANGE, simplified=False)

    th = hurst[100:].mean()
    th_eps = hurst[100:].std() * 1e-3
    th_eps = hurst[100:].std() * 3.5e-1

    frame['Persistent'] = (hurst > th + th_eps).astype(np.int32)
    frame['Reversal'] = (hurst < th - th_eps).astype(np.int32)
    frame['RandomWalk'] = (np.abs(hurst - th) < th_eps).astype(np.int32)
    frame['i_Hurst'] = hurst
    return frame

frame = preprocess(frame)
val_frame = preprocess(val_frame)

  0%|          | 0/9911 [00:00<?, ?it/s]

  0%|          | 0/9791 [00:00<?, ?it/s]

In [64]:
from jellyfish.transform.sampling import _generic_sampling, DEFAULT_SAMPLING_AGG
from jellyfish.core import SmaCross, Backtest

condition = lambda ohlc: ohlc.Persistent.max() + ohlc.Reversal.max() > 1
frame1 = _generic_sampling(frame.reset_index(), condition, DEFAULT_SAMPLING_AGG).set_index('Date')
frame2 = _generic_sampling(val_frame.reset_index(), condition, DEFAULT_SAMPLING_AGG).set_index('Date')

  0%|          | 0/10010 [00:00<?, ?it/s]

  0%|          | 0/9890 [00:00<?, ?it/s]

In [65]:
bt = Backtest(frame1, SmaCross, trade_on_close=True)
stats = bt.optimize(
    n1=range(5, 25, 2),
    n2=range(5,25, 2),
    constraint=lambda p: p.n1 < p.n2,
    maximize='Calmar Ratio',
)
# bt.plot(open_browser=True)
stats

Backtest.optimize:   0%|          | 0/15 [00:00<?, ?it/s]

Start                     2021-09-18 06:08:00
End                       2021-09-25 03:00:00
Duration                      6 days 20:52:00
Exposure Time [%]                   94.690265
Equity Final [$]               1167017.928536
Equity Peak [$]                1172234.173016
Return [%]                          16.701793
Buy & Hold Return [%]              -10.295659
Return (Ann.) [%]               105524.718518
Volatility (Ann.) [%]            72687.003483
Sharpe Ratio                        10.374384
Calmar Ratio                       175.602478
Max. Drawdown [%]                   -4.887543
Avg. Drawdown [%]                   -1.552991
Max. Drawdown Duration        2 days 13:41:00
Avg. Drawdown Duration        0 days 13:58:00
# Trades                                   35
Win Rate [%]                        48.571429
Best Trade [%]                       7.290834
Worst Trade [%]                     -2.046613
Avg. Trade [%]                       0.448813
Max. Trade Duration           0 da

In [66]:
params = stats._strategy
bt = Backtest(frame2, SmaCross, trade_on_close=True)
stats = bt.run(
    n1=params.n1,
    n2=params.n2
)
bt.plot(open_browser=True)
stats

Start                     2021-09-25 04:45:00
End                       2021-10-02 01:40:00
Duration                      6 days 20:55:00
Exposure Time [%]                   95.594714
Equity Final [$]               1043418.599184
Equity Peak [$]                1044551.759184
Return [%]                            4.34186
Buy & Hold Return [%]               13.437118
Return (Ann.) [%]                 1326.935403
Volatility (Ann.) [%]             1016.010806
Sharpe Ratio                         3.790872
Calmar Ratio                        28.802696
Max. Drawdown [%]                   -8.608719
Avg. Drawdown [%]                   -2.438393
Max. Drawdown Duration        5 days 04:35:00
Avg. Drawdown Duration        1 days 07:40:00
# Trades                                   38
Win Rate [%]                        42.105263
Best Trade [%]                       8.982103
Worst Trade [%]                     -2.030996
Avg. Trade [%]                       0.118545
Max. Trade Duration           0 da